# LLM: Zero-shot classification through LLMs and prompts

Classify data into categories without being explicitly trained on labeled examples for that specific task. Instead, the model uses pre-trained knowledge and natural language prompts to infer the correct category.

| Prompt Name              | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|--------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **simple_prompt**        | "Does the following person develop a psychological disorder between time point T1 and T2? Data: …"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| **class_definitions_prompt** | "Given the following data, classify whether this person develops a psychological disorder between T1 and T2 according to the instructions provided and data measured by F-DIPS structural interviews. Respond with YES or NO. Instructions: NO: The person did not develop any new psychological disorder between T1 and T2. This means they were either healthy at both time points, had an ongoing disorder across both time points, or had already recovered from a previous disorder. YES: The person was psychologically healthy at T1 but developed a psychological disorder at T2. Data: …" |
| **profiled_simple_prompt** | "You are an expert in clinical psychology and mental health diagnostics. You are trained to analyze longitudinal data to assess whether a person develops a psychological disorder over time. You know how to analyze data measured with F-DIPS structural interviews at two time points T1 and T2. Tell me if the following data shows the development of a psychological disorder between time point T1 and T2. Just answer YES or NO. Data: …"                                                                                                                                             |
| **few_shot_prompt**      | "Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: … Based on the previous example data prompts, classify the following data. Does the following person develop a psychological disorder between time point T1 and T2? Data: …"                                                                                                                                                                                                                                   |
| **vignette_text_prompt** | "Data is in csv format, but what if we prompt ChatGPT to convert the table format into a vivid/interesting/emotional text? → vignette. Vignette: Each fictional patient was assigned an age, sex, physical examination findings, and clinical symptoms. ‘Can you grade this case according to … Classification System?’"                                                                                                                                                                                                                                                                      |

## 0 Imports

In [1]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import pdist

In [2]:
data = pd.read_csv("../dat/dips/DIPS_Data_cleaned.csv", sep = ",", low_memory = False)
data_pred = pd.read_csv("../dat/dips/DIPS_Data_cleaned_pred.csv", sep = ",", low_memory = False)
data_pred_y = pd.read_csv("../dat/dips/DIPS_Data_cleaned_pred_y.csv", sep = ",", low_memory = False)

## 1 Zero-shot classification

### 1.1 Data preparation

In [3]:
# Predictors
X = data
X = X.drop(["hpi"], axis = 1)

# Target
y = data["hpi"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# concatenate X and y for example data
dat_train = pd.concat([X_train, y_train], axis = 1)

print("LLMs \n",
      "X_train shape: ", X_train.shape, round(X_train.shape[0]/len(X), 2), "\n",
      "X_test shape: ", X_test.shape, round(X_test.shape[0]/len(X), 2),  "\n",
      "y_train shape: ", y_train.shape, round(y_train.shape[0]/len(y), 2), "\n",
      "y_test shape: ", y_test.shape, round(y_test.shape[0]/len(y), 2), "\n")

LLMs 
 X_train shape:  (1005, 22) 0.8 
 X_test shape:  (252, 22) 0.2 
 y_train shape:  (1005,) 0.8 
 y_test shape:  (252,) 0.2 



In [4]:
# get clear names for each column in data
preds = X.columns
preds

Index(['whi_ges', 'soz_ges', 'gke_ges', 'lzh_ges', 'woc_gesp', 'woc_gese',
       'asi_ges', 'bsq_ges', 'das_ges', 'scl_gsi', 'bild', 'bmi_kat',
       'ses_kom', 'whi_gesy', 'soz_gesy', 'gke_gesy', 'lzh_gesy', 'asi_gesy',
       'bsq_gesy', 'das_gesy', 'scl_gsiy', 'ile'],
      dtype='object')

In [5]:
predictor_names = ["T1 Positive mental health",    #whi_ges
                   "T1 Social support",            #soz_ges (SAVE)
                   "T1 General self-efficacy",     #gke_ges SAVE
                   "T1 Life satisfaction",         #lzh_ges SAVE
                   "T1 Problem-focused coping",    #woc_gesp SAVE
                   "T1 Emotion-focused coping",    #woc_gese SAVE
                   "T1 Anxiety sensitivity",       #asi_ges SAVE
                   "T1 Fear of bodily sensations", #bsq_ges SAVE
                   "T1 Dysfunctional attitudes",   #das_ges SAVE
                   "T1 General psychopathology: Global Severity Index (GSI)",    #scl_gsi SAVE
                   "Education",                    #bild
                   "T1 BMI",                          #bmi_kat
                   "Socioeconomic status",         #ses_kom
                   "T2 Positive mental health",    #whi_gesy
                   "T2 Social support",            #soz_gesy (SAVE)
                   "T2 General self-efficacy",     #gke_gesy SAVE
                   "T2 Life satisfaction",         #lzh_gesy SAVE
                   "T2 Anxiety sensitivity",       #asi_gesy SAVE
                   "T2 Fear of bodily sensations", #bsq_gesy SAVE
                   "T2 Dysfunctional attitudes",   #das_gesy SAVE
                   "T2 General psychopathology: Global Severity Index (GSI)",    #scl_gsi SAVE
                   "T1 Stress"                     #ile SAVE
                   ]

target_names = ["Total mental disorders incidence point prevalence"]    #hpi

In [6]:
# print col and names
for col, name in zip(preds, predictor_names):
    print(f"{col}: {name}")

whi_ges: T1 Positive mental health
soz_ges: T1 Social support
gke_ges: T1 General self-efficacy
lzh_ges: T1 Life satisfaction
woc_gesp: T1 Problem-focused coping
woc_gese: T1 Emotion-focused coping
asi_ges: T1 Anxiety sensitivity
bsq_ges: T1 Fear of bodily sensations
das_ges: T1 Dysfunctional attitudes
scl_gsi: T1 General psychopathology: Global Severity Index (GSI)
bild: Education
bmi_kat: T1 BMI
ses_kom: Socioeconomic status
whi_gesy: T2 Positive mental health
soz_gesy: T2 Social support
gke_gesy: T2 General self-efficacy
lzh_gesy: T2 Life satisfaction
asi_gesy: T2 Anxiety sensitivity
bsq_gesy: T2 Fear of bodily sensations
das_gesy: T2 Dysfunctional attitudes
scl_gsiy: T2 General psychopathology: Global Severity Index (GSI)
ile: T1 Stress


#### 1.1 Testing prompting

In [7]:
client = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY"),
)

# testing
response = client.responses.create(
    model = "gpt-4o-mini",
    instructions = "You are a coding assistant that talks like a pirate.",
    input = "How do I check if a Python object is an instance of a class?",
)

print(response.output_text)

Ahoy, matey! If ye be wantin' to check if a Python object be an instance of a certain class, ye can use the `isinstance()` function. Here be the syntax ye be needin':

```python
isinstance(yar_object, YourClass)
```

Replace `yar_object` with the object ye wish to check, and `YourClass` with the class ye be investigatin'. This here function will return `True` if the object be an instance of the specified class, or `False` if it be not. 

Here be an example fer ye:

```python
class Pirate:
    pass

capn = Pirate()

if isinstance(capn, Pirate):
    print("Aye, be a true pirate!")
else:
    print("This be no pirate!")
```

And that be how ye check if a scallywag be of a particular class! Arrr! 🏴‍☠️


### 1.2 Simple prompt

In [9]:
def create_simple_prompt(data_row):
    """
    Create a simple prompt for zero-shot classification.
    """

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Respond with YES or NO."

    prompt = f"Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    return instruction, prompt

In [10]:
# Test the function
data_row = X.iloc[4]
simple_instrcution, simple_prompt = create_simple_prompt(data_row)
print(simple_prompt)
print("\n", target_names[0], y.iloc[4])

Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.0346394734930797, T1 Social support: -1.9443495599450165, T1 General self-efficacy: 0.5865222429197926, T1 Life satisfaction: -1.2777276005686404, T1 Problem-focused coping: -0.6434380872019897, T1 Emotion-focused coping: -0.4501970947608277, T1 Anxiety sensitivity: -1.6912121763530454, T1 Fear of bodily sensations: -1.3399218609334862, T1 Dysfunctional attitudes: 1.528535706802646, T1 General psychopathology: Global Severity Index (GSI): -0.5273858134379611, Education: 1.0, T1 BMI: 1.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.2748340578894306, T2 Social support: 1.0533586187543231, T2 General self-efficacy: 1.3559331135757966, T2 Life satisfaction: 1.005628346789385, T2 Anxiety sensitivity: -1.1666271173074825, T2 Fear of bodily sensations: -1.0416157340293282, T2 Dysfunctional attitudes: -0.8267579141727915, T2 General psychopathology: Global S

In [11]:
# Create a simple prompt for each row in the test set
X_test_simple_prompt = []

for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    simple_instrcution, simple_prompt = create_simple_prompt(data_row)
    X_test_simple_prompt.append(simple_prompt)

X_test_simple_prompt[0]

'Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.9916025104337652, T1 Social support: 0.1912554943776842, T1 General self-efficacy: -1.209478876081861, T1 Life satisfaction: 0.1815598805259552, T1 Problem-focused coping: -1.0834346466693496, T1 Emotion-focused coping: -0.1263180001181833, T1 Anxiety sensitivity: 1.083708747266506, T1 Fear of bodily sensations: 1.2360549695511902, T1 Dysfunctional attitudes: -0.6885415900752596, T1 General psychopathology: Global Severity Index (GSI): -0.4917971179478549, Education: 2.0, T1 BMI: 2.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.7828728972526869, T2 Social support: 0.0984652458135236, T2 General self-efficacy: -1.1421997527016898, T2 Life satisfaction: -0.6556858132553557, T2 Anxiety sensitivity: 1.5262249302236135, T2 Fear of bodily sensations: 1.9574857842600144, T2 Dysfunctional attitudes: -0.2807869674967928, T2 General psychopathology: Global Se

In [12]:
# create random input for testing
data_row = np.random.rand(len(predictor_names))
simple_instrcution, simple_prompt = create_simple_prompt(data_row)
print(simple_prompt)

Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: 0.7507372176265392, T1 Social support: 0.11735830522172208, T1 General self-efficacy: 0.49252118340805673, T1 Life satisfaction: 0.9497756586464509, T1 Problem-focused coping: 0.9652559280386542, T1 Emotion-focused coping: 0.9368693123418592, T1 Anxiety sensitivity: 0.7994326061675615, T1 Fear of bodily sensations: 0.4273534035217079, T1 Dysfunctional attitudes: 0.5502422548092575, T1 General psychopathology: Global Severity Index (GSI): 0.49389367728464706, Education: 0.6623771672979972, T1 BMI: 0.32292273892592505, Socioeconomic status: 0.9028435558967834, T2 Positive mental health: 0.35749626959476744, T2 Social support: 0.08626136622832081, T2 General self-efficacy: 0.6958579564643403, T2 Life satisfaction: 0.5831484913681354, T2 Anxiety sensitivity: 0.5418005658089129, T2 Fear of bodily sensations: 0.6059446790650808, T2 Dysfunctional attitudes: 0.3116764498009

In [13]:
# client = OpenAI(
#     api_key = os.environ.get("OPENAI_API_KEY"),
# )
#
# # testing
# response = client.responses.create(
#     model = "gpt-4o-mini",
#     instructions = simple_instrcution,
#     input = simple_prompt,
# )
#
# print(response.output_text)
#
# # save the response in array
# response_array = []
# response_array.append(response.output_text)

YES


In [20]:
# simple_prompt_array = []
#
# # iterate over the test set and save the response for each prompt in an array
# for prompt in X_test_simple_prompt:
#     response = client.responses.create(
#         model = "gpt-4o",
#         instructions = simple_instrcution,
#         input = prompt,
#     )
#     simple_prompt_array.append(response.output_text)
#
# # value counts for array
# counts_simple = pd.Series(simple_prompt_array).value_counts()
# print(counts_simple)
#
# # convert YES to 1 and NO to 0
# simple_prompt_array = [1 if response == "YES" else 0 for response in simple_prompt_array]
# simple_prompt_array

[1]

### 1.3 Class definitions prompt

In [31]:
def create_class_definitions_prompt(data_row):
    """
    Create a class definitions prompt for zero-shot classification.
    """

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Respond with YES or NO."

    prompt = f"Given the following data, classify whether this person develops a psychological disorder between T1 and T2 according to the instructions provided and data measured by F-DIPS structural interviews. Respond with YES or NO. Instructions: NO: The person did not develop any new psychological disorder between T1 and T2. This means they were either healthy at both time points, had an ongoing disorder across both time points, or had already recovered from a previous disorder. YES: The person was psychologically healthy at T1 but developed a psychological disorder at T2. Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    return instruction, prompt

In [32]:
# Test the function
data_row = X.iloc[4]
class_definitions_instruction, class_definitions_prompt = create_class_definitions_prompt(data_row)
print(class_definitions_prompt)
print("\n", target_names[0], y.iloc[4])

Given the following data, classify whether this person develops a psychological disorder between T1 and T2 according to the instructions provided and data measured by F-DIPS structural interviews. Respond with YES or NO. Instructions: NO: The person did not develop any new psychological disorder between T1 and T2. This means they were either healthy at both time points, had an ongoing disorder across both time points, or had already recovered from a previous disorder. YES: The person was psychologically healthy at T1 but developed a psychological disorder at T2. Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.0346394734930797, T1 Social support: -1.9443495599450165, T1 General self-efficacy: 0.5865222429197926, T1 Life satisfaction: -1.2777276005686404, T1 Problem-focused coping: -0.6434380872019897, T1 Emotion-focused coping: -0.4501970947608277, T1 Anxiety sensitivity: -1.6912121763530454, T1 Fear of bodily 

In [33]:
# Create a class definitions prompt for each row in the test set
X_test_class_definitions_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    class_definitions_instruction, class_definitions_prompt = create_class_definitions_prompt(data_row)
    X_test_class_definitions_prompt.append(class_definitions_prompt)

X_test_class_definitions_prompt[0]

'Given the following data, classify whether this person develops a psychological disorder between T1 and T2 according to the instructions provided and data measured by F-DIPS structural interviews. Respond with YES or NO. Instructions: NO: The person did not develop any new psychological disorder between T1 and T2. This means they were either healthy at both time points, had an ongoing disorder across both time points, or had already recovered from a previous disorder. YES: The person was psychologically healthy at T1 but developed a psychological disorder at T2. Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.9916025104337652, T1 Social support: 0.1912554943776842, T1 General self-efficacy: -1.209478876081861, T1 Life satisfaction: 0.1815598805259552, T1 Problem-focused coping: -1.0834346466693496, T1 Emotion-focused coping: -0.1263180001181833, T1 Anxiety sensitivity: 1.083708747266506, T1 Fear of bodily sen

In [34]:
# create random input for testing
data_row = np.random.rand(len(predictor_names))
class_definitions_instruction, class_definitions_prompt = create_class_definitions_prompt(data_row)
print(class_definitions_prompt)

Given the following data, classify whether this person develops a psychological disorder between T1 and T2 according to the instructions provided and data measured by F-DIPS structural interviews. Respond with YES or NO. Instructions: NO: The person did not develop any new psychological disorder between T1 and T2. This means they were either healthy at both time points, had an ongoing disorder across both time points, or had already recovered from a previous disorder. YES: The person was psychologically healthy at T1 but developed a psychological disorder at T2. Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: 0.8749287774490059, T1 Social support: 0.7610825061439005, T1 General self-efficacy: 0.18047766908148144, T1 Life satisfaction: 0.1890946042584577, T1 Problem-focused coping: 0.41579402028979273, T1 Emotion-focused coping: 0.9860875602298624, T1 Anxiety sensitivity: 0.0009149475325579415, T1 Fear of bodily s

In [35]:
# class_def_array = []
#
# # iterate over the test set and save the response for each prompt in an array
# for prompt in X_test_class_definitions_prompt:
#     response = client.responses.create(
#         model = "gpt-4o",
#         instructions = class_definitions_instruction,
#         input = prompt,
#     )
#     class_def_array.append(response.output_text)
#
# # value counts for array
# counts_class_def = pd.Series(class_def_array).value_counts()
# print(counts_class_def)
#
# # convert YES to 1 and NO to 0
# class_def_array = [1 if response == "YES" else 0 for response in class_def_array]
# class_def_array

YES    1
Name: count, dtype: int64


[1]

### 1.4 Profiled simple prompt

In [38]:
def create_profiled_simple_prompt(data_row):
    """
    Create a profiled simple prompt for zero-shot classification.
    """

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Respond with YES or NO."

    prompt = f"You are an expert in clinical psychology and mental health diagnostics. You are trained to analyze longitudinal data to assess whether a person develops a psychological disorder over time. You know how to analyze data measured with F-DIPS structural interviews at two time points T1 and T2. Tell me if the following data shows the development of a psychological disorder between time point T1 and T2. Just answer YES or NO. Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    return instruction, prompt

In [39]:
# Test the function
data_row = X.iloc[4]
profiled_simple_instruction, profiled_simple_prompt = create_profiled_simple_prompt(data_row)
print(profiled_simple_prompt)
print("\n", target_names[0], y.iloc[4])

You are an expert in clinical psychology and mental health diagnostics. You are trained to analyze longitudinal data to assess whether a person develops a psychological disorder over time. You know how to analyze data measured with F-DIPS structural interviews at two time points T1 and T2. Tell me if the following data shows the development of a psychological disorder between time point T1 and T2. Just answer YES or NO. Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.0346394734930797, T1 Social support: -1.9443495599450165, T1 General self-efficacy: 0.5865222429197926, T1 Life satisfaction: -1.2777276005686404, T1 Problem-focused coping: -0.6434380872019897, T1 Emotion-focused coping: -0.4501970947608277, T1 Anxiety sensitivity: -1.6912121763530454, T1 Fear of bodily sensations: -1.3399218609334862, T1 Dysfunctional attitudes: 1.528535706802646, T1 General psychopathology: Global Severity Index (GSI): -0.52738

In [40]:
# Create a profiled simple prompt for each row in the test set
X_test_profiled_simple_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    profiled_simple_instruction, profiled_simple_prompt = create_profiled_simple_prompt(data_row)
    X_test_profiled_simple_prompt.append(profiled_simple_prompt)

X_test_profiled_simple_prompt[0]

'You are an expert in clinical psychology and mental health diagnostics. You are trained to analyze longitudinal data to assess whether a person develops a psychological disorder over time. You know how to analyze data measured with F-DIPS structural interviews at two time points T1 and T2. Tell me if the following data shows the development of a psychological disorder between time point T1 and T2. Just answer YES or NO. Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: -0.9916025104337652, T1 Social support: 0.1912554943776842, T1 General self-efficacy: -1.209478876081861, T1 Life satisfaction: 0.1815598805259552, T1 Problem-focused coping: -1.0834346466693496, T1 Emotion-focused coping: -0.1263180001181833, T1 Anxiety sensitivity: 1.083708747266506, T1 Fear of bodily sensations: 1.2360549695511902, T1 Dysfunctional attitudes: -0.6885415900752596, T1 General psychopathology: Global Severity Index (GSI): -0.4917971

In [41]:
# create random input for testing
data_row = np.random.rand(len(predictor_names))
profiled_simple_instruction, profiled_simple_prompt = create_profiled_simple_prompt(data_row)
print(profiled_simple_prompt)

You are an expert in clinical psychology and mental health diagnostics. You are trained to analyze longitudinal data to assess whether a person develops a psychological disorder over time. You know how to analyze data measured with F-DIPS structural interviews at two time points T1 and T2. Tell me if the following data shows the development of a psychological disorder between time point T1 and T2. Just answer YES or NO. Does the following person develop a psychological disorder between time point T1 and T2? Data: T1 Positive mental health: 0.8369216825683595, T1 Social support: 0.35453389906903376, T1 General self-efficacy: 0.7357594549603136, T1 Life satisfaction: 0.3848814785241943, T1 Problem-focused coping: 0.5480607842359527, T1 Emotion-focused coping: 0.6965397522845032, T1 Anxiety sensitivity: 0.217246243124411, T1 Fear of bodily sensations: 0.5892875705151893, T1 Dysfunctional attitudes: 0.8010613980525763, T1 General psychopathology: Global Severity Index (GSI): 0.515265643376

array([0.83692168, 0.3545339 , 0.73575945, 0.38488148, 0.54806078,
       0.69653975, 0.21724624, 0.58928757, 0.8010614 , 0.51526564,
       0.54802605, 0.57983128, 0.4625416 , 0.12408278, 0.6661791 ,
       0.88243884, 0.43210819, 0.56377408, 0.24199405, 0.23841351,
       0.99795431, 0.17574   ])

In [42]:
# profiled_simple_array = []
#
# # iterate over the test set and save the response for each prompt in an array
# for prompt in X_test_profiled_simple_prompt:
#     response = client.responses.create(
#         model = "gpt-4o",
#         instructions = profiled_simple_instruction,
#         input = prompt,
#     )
#     profiled_simple_array.append(response.output_text)
#
# # value counts for array
# counts_profiled_simple = pd.Series(profiled_simple_array).value_counts()
# print(counts_profiled_simple)
#
# # convert YES to 1 and NO to 0
# profiled_simple_array = [1 if response == "YES" else 0 for response in profiled_simple_array]
# profiled_simple_array

YES    1
Name: count, dtype: int64


[1]

### 1.5 Few-shot prompt

In [43]:
def create_few_shot_prompt(data_row, few_shot_examples):
    """
    Create a few-shot prompt for zero-shot classification.
    """

    example_data = []

    # create example prompt but including the target aswell, stating Example 1: ..., Example 2: ...
    for i, example in enumerate(few_shot_examples):
        example_data.append(
            f"Example {i + 1}: " + ", ".join([f"{name}: {value}" for name, value in zip(predictor_names + target_names, example)])
        )

    # Convert the data row to a string format using the real column names
    data_string = ", ".join([f"{name}: {value}" for name, value in zip(predictor_names, data_row)])

    instruction = "Respond with YES or NO."

    prompt1 = f"Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: {example_data}"

    prompt2 = f"Based on the previous example data prompts, classify the following data. Does the following person develop a psychological disorder between time point T1 and T2? Data: {data_string}"

    # concatenate
    prompt = f"{prompt1} {prompt2}"

    return instruction, prompt

In [44]:
def get_similar_examples(data_row, dat_train, n):
    """
    Get the top-n semantically similar examples to the test product from the training set.
    """

    # get columns that we want to use for similarity
    preds = dat_train.drop("hpi", axis = 1).columns
    dat_preds = dat_train[preds]
    data_row = data_row[preds]
    data_row = np.array([data_row])

    # dictionary to store the distances
    distances = {}

    for row in dat_preds.iterrows():
        # create array with the row and the data_row
        row_input = np.array([row[1]])
        dist_row = np.concatenate((data_row, row_input), axis = 0)

        # calculate the distance between the two rows
        dist = pdist(dist_row, metric = "euclidean")

        # store the distance in the dictionary
        distances[row[0]] = dist[0]

    # sort the dictionary by distance
    sorted_distances = sorted(distances.items(), key = lambda x: x[1])

    # get top keys
    keys = []
    for key in sorted_distances[:n]:
        keys.append(key[0])

    # drop all rows that are not in keys
    ex_dat = dat_train.loc[keys]

    return ex_dat

In [46]:
# Test the function
close_ex = get_similar_examples(X.iloc[4], dat_train, 5)

data_row = X.iloc[4]
few_shot_examples = close_ex.values
few_shot_instruction, few_shot_prompt = create_few_shot_prompt(data_row, few_shot_examples)
print(few_shot_prompt)

Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: ['Example 1: T1 Positive mental health: -0.0346394734930797, T1 Social support: -1.9443495599450165, T1 General self-efficacy: 0.5865222429197926, T1 Life satisfaction: -1.2777276005686404, T1 Problem-focused coping: -0.6434380872019897, T1 Emotion-focused coping: -0.4501970947608277, T1 Anxiety sensitivity: -1.6912121763530454, T1 Fear of bodily sensations: -1.3399218609334862, T1 Dysfunctional attitudes: 1.528535706802646, T1 General psychopathology: Global Severity Index (GSI): -0.5273858134379611, Education: 1.0, T1 BMI: 1.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.2748340578894306, T2 Social support: 1.0533586187543231, T2 General self-efficacy: 1.3559331135757966, T2 Life satisfaction: 1.005628346789385, T2 Anxiety sensitivity: -1.1666271173074825, T2 Fear of bodily sensations: -1.0416157340293282, T2 D

In [47]:
# Create a few-shot prompt for each row in the test set
X_test_few_shot_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    few_shot_examples = get_similar_examples(data_row, dat_train, 5).values
    few_shot_instruction, few_shot_prompt = create_few_shot_prompt(data_row, few_shot_examples)
    X_test_few_shot_prompt.append(few_shot_prompt)

X_test_few_shot_prompt[0]

"Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: ['Example 1: T1 Positive mental health: -0.5131209919634224, T1 Social support: -0.0756951374126611, T1 General self-efficacy: -0.0869781767058276, T1 Life satisfaction: 0.9922751478007364, T1 Problem-focused coping: -0.9074360228824055, T1 Emotion-focused coping: -0.4501970947608277, T1 Anxiety sensitivity: 1.083708747266506, T1 Fear of bodily sensations: 1.6176811666600328, T1 Dysfunctional attitudes: -0.3957200602989325, T1 General psychopathology: Global Severity Index (GSI): 0.1487994008740618, Education: 2.0, T1 BMI: 2.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.2748340578894306, T2 Social support: -0.1880027660686982, T2 General self-efficacy: -0.915096764858282, T2 Life satisfaction: -0.8174066606933537, T2 Anxiety sensitivity: 2.1673801796357792, T2 Fear of bodily sensations: 2.344466625329608, T2 Dy

In [77]:
# create random input for testing
data_row = X_test.iloc[4]
# add a random number to the data_row
data_row = data_row + np.random.rand(len(data_row))

few_shot_examples = get_similar_examples(data_row, dat_train, 5).values
few_shot_instruction, few_shot_prompt = create_few_shot_prompt(data_row, few_shot_examples)
print(few_shot_prompt)

Please refer to the following examples of correctly classified data points with 'Total mental disorders incidence point prevalence' being the true classification: ['Example 1: T1 Positive mental health: -0.5131209919634224, T1 Social support: 0.8141403018884763, T1 General self-efficacy: 0.5865222429197926, T1 Life satisfaction: -0.30486927983891, T1 Problem-focused coping: 0.852550214987034, T1 Emotion-focused coping: -1.0169855103854553, T1 Anxiety sensitivity: -0.2376821687428043, T1 Fear of bodily sensations: -0.2904498188841784, T1 Dysfunctional attitudes: -0.9395314727406828, T1 General psychopathology: Global Severity Index (GSI): -0.0291440765764721, Education: 2.0, T1 BMI: 2.0, Socioeconomic status: 2.0, T2 Positive mental health: -0.2748340578894306, T2 Social support: 0.6714012695780124, T2 General self-efficacy: 1.3559331135757966, T2 Life satisfaction: 0.3146392713725531, T2 Anxiety sensitivity: -0.5254718678953167, T2 Fear of bodily sensations: -0.4611444724249378, T2 Dys

In [78]:
# few_shot_array = []
#
# # iterate over the test set and save the response for each prompt in an array
# for prompt in X_test_few_shot_prompt:
#     response = client.responses.create(
#         model = "gpt-4o",
#         instructions = few_shot_instruction,
#         input = prompt,
#     )
#     few_shot_array.append(response.output_text)
#
# # value counts for array
# counts_few_shot = pd.Series(few_shot_array).value_counts()
# print(counts_few_shot)
#
# # convert YES to 1 and NO to 0
# few_shot_array = [1 if response == "YES" else 0 for response in few_shot_array]
# few_shot_array

NO    1
Name: count, dtype: int64


[0]

### 1.6 Vignette text prompt

In [79]:
def interpret_value(val, thresh_low = -0.25, thresh_high = 0.25):
    """Convert standardized value to qualitative description with value."""
    val_str = f"{val:.3f}"
    if val > thresh_high:
        return f"above average ({val_str})"
    elif val < thresh_low:
        return f"below average ({val_str})"
    else:
        return f"average ({val_str})"

In [83]:
def create_vignette_prompt(data):

    # demographics
    bmi = data['bmi_kat']
    education = data['bild']
    ses = data['ses_kom']

    # time point 1
    t1_pm_health = interpret_value(data['whi_ges'])
    t1_support = interpret_value(data['soz_ges'])
    t1_self_eff = interpret_value(data['gke_ges'])
    t1_life_sat = interpret_value(data['lzh_ges'])
    t1_coping_prob = interpret_value(data['woc_gesp'])
    t1_coping_emot = interpret_value(data['woc_gese'])
    t1_anx_sens = interpret_value(data['asi_ges'])
    t1_fear = interpret_value(data['bsq_ges'])
    t1_dysfunc = interpret_value(data['das_ges'])
    t1_gsi = interpret_value(data['scl_gsi'])
    t1_stress = interpret_value(data['ile'])

    # time point 2
    t2_pm_health = interpret_value(data['whi_gesy'])
    t2_support = interpret_value(data['soz_gesy'])
    t2_self_eff = interpret_value(data['gke_gesy'])
    t2_life_sat = interpret_value(data['lzh_gesy'])
    t2_anx_sens = interpret_value(data['asi_gesy'])
    t2_fear = interpret_value(data['bsq_gesy'])
    t2_dysfunc = interpret_value(data['das_gesy'])
    t2_gsi = interpret_value(data['scl_gsiy'])

    instruction = "Respond with YES or NO."

    # generate prompt
    vignette = f"""A woman with a BMI of {bmi}, an education level of {education}, and a socioeconomic status of {ses} has the following psychological profile: At Time Point 1, she showed {t1_pm_health} positive mental health, {t1_support} social support, and {t1_self_eff} general self-efficacy. Her life satisfaction was {t1_life_sat}, and she relied on {t1_coping_prob} problem-focused and {t1_coping_emot} emotion-focused coping strategies. Anxiety sensitivity was {t1_anx_sens}, and her fear of bodily sensations was {t1_fear}. She reported {t1_dysfunc} levels of dysfunctional attitudes and {t1_gsi} levels of general psychopathology. Her stress level was {t1_stress}. By Time Point 2, approximately 17 months later, she reported {t2_pm_health} positive mental health, {t2_support} social support, and {t2_self_eff} self-efficacy. Life satisfaction remained {t2_life_sat}. However, anxiety sensitivity had become {t2_anx_sens}, and fear of bodily sensations was {t2_fear}. Dysfunctional attitudes were {t2_dysfunc}, and general psychopathology was {t2_gsi}. Does this person develop a psychological disorder between time point T1 and T2?"""
    return instruction, vignette

In [84]:
# Test the function
data_row = X.iloc[4]
vignette_instruction, vignette_prompt = create_vignette_prompt(data_row)
print(vignette_prompt)

A woman with a BMI of 1.0, an education level of 1.0, and a socioeconomic status of 2.0 has the following psychological profile: At Time Point 1, she showed average (-0.035) positive mental health, below average (-1.944) social support, and above average (0.587) general self-efficacy. Her life satisfaction was below average (-1.278), and she relied on below average (-0.643) problem-focused and below average (-0.450) emotion-focused coping strategies. Anxiety sensitivity was below average (-1.691), and her fear of bodily sensations was below average (-1.340). She reported above average (1.529) levels of dysfunctional attitudes and below average (-0.527) levels of general psychopathology. Her stress level was below average (-0.363). By Time Point 2, approximately 17 months later, she reported below average (-0.275) positive mental health, above average (1.053) social support, and above average (1.356) self-efficacy. Life satisfaction remained above average (1.006). However, anxiety sensi

In [85]:
# Create a vignette prompt for each row in the test set
X_test_vignette_prompt = []
for i in range(len(X_test)):
    data_row = X_test.iloc[i]
    vignette_instruction, vignette_prompt = create_vignette_prompt(data_row)
    X_test_vignette_prompt.append(vignette_prompt)

X_test_vignette_prompt[0]

'A woman with a BMI of 2.0, an education level of 2.0, and a socioeconomic status of 2.0 has the following psychological profile: At Time Point 1, she showed below average (-0.992) positive mental health, average (0.191) social support, and below average (-1.209) general self-efficacy. Her life satisfaction was average (0.182), and she relied on below average (-1.083) problem-focused and average (-0.126) emotion-focused coping strategies. Anxiety sensitivity was above average (1.084), and her fear of bodily sensations was above average (1.236). She reported below average (-0.689) levels of dysfunctional attitudes and below average (-0.492) levels of general psychopathology. Her stress level was below average (-0.763). By Time Point 2, approximately 17 months later, she reported below average (-0.783) positive mental health, average (0.098) social support, and below average (-1.142) self-efficacy. Life satisfaction remained below average (-0.656). However, anxiety sensitivity had become

In [86]:
# create random input for testing
data_row = X_test.iloc[4]
# add a random number to the data_row
data_row = data_row + np.random.rand(len(data_row))

vignette_instruction, vignette_prompt = create_vignette_prompt(data_row)
print(vignette_prompt)

A woman with a BMI of 2.9639670710033856, an education level of 1.5633637636280175, and a socioeconomic status of 1.2759668460855185 has the following psychological profile: At Time Point 1, she showed below average (-0.982) positive mental health, above average (1.991) social support, and above average (0.966) general self-efficacy. Her life satisfaction was average (-0.153), and she relied on above average (1.690) problem-focused and above average (0.692) emotion-focused coping strategies. Anxiety sensitivity was above average (0.468), and her fear of bodily sensations was below average (-0.405). She reported above average (0.600) levels of dysfunctional attitudes and below average (-0.460) levels of general psychopathology. Her stress level was above average (1.532). By Time Point 2, approximately 17 months later, she reported average (0.056) positive mental health, above average (1.588) social support, and above average (0.414) self-efficacy. Life satisfaction remained below averag

In [87]:
# vignette_array = []
#
# # iterate over the test set and save the response for each prompt in an array
# for prompt in X_test_vignette_prompt:
#     response = client.responses.create(
#         model = "gpt-4o",
#         instructions = vignette_instruction,
#         input = prompt,
#     )
#     vignette_array.append(response.output_text)
#
# # value counts for array
# counts_vignette = pd.Series(vignette_array).value_counts()
# print(counts_vignette)
#
# # convert YES to 1 and NO to 0
# vignette_array = [1 if response == "YES" else 0 for response in vignette_array]
# vignette_array

YES    1
Name: count, dtype: int64


[1]